In [0]:
import dlt
from pyspark.sql.functions import col, expr

In [0]:
ambiente = 'dev'

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS dados.teste;
-- DROP DATABASE IF EXISTS dados.teste;

In [0]:
df_products = spark.read.format("csv").option("header", "true").load("/FileStore/tmp/testes/products_data.csv")
display(df_products)

In [0]:
@dlt.table(
    name="bronze_products",
    comment="Bronze products",
    table_properties={"quality": "bronze"}
)
def bronze_products():
  df_bronze = spark.read.format("csv").option("header", "true").load("/FileStore/tmp/testes/sales_data.csv")
  return df_bronze

In [0]:
@dlt.table(
  name="silver_products",
  comment="Silver products",
  table_properties={"quality": "silver"}
)
def silver_products():
  df_silver = dlt.read_stream("bronze_teste")
  df_cleaned = df_silver.filter(col("sales_amount").isNotNull()) \
                    .withColumn("sales_amount", col("sales_amount").cast("float")) \
                    .withColumn("sale_date", expr("to_date(sale_date, 'yyyy-MM-dd')"))
  return df_cleaned

In [0]:
@dlt.table(
  name="gold_products",
  comment="Gold products",
  table_properties={"quality": "gold"}
)
def gold_products():
  df_gold = dlt.read_stream("silver_teste")
  df_summary = df_gold.groupBy("sale_date") \
                   .agg(
                       expr("sum(sales_amount)").alias("total_sales"),
                       expr("count(*)").alias("total_transactions")
                   )
  return df_summary